# ChatModelクラスとLLMクラスの違い

In [10]:
from langchain_google_vertexai import ChatVertexAI
chat_model = ChatVertexAI(project="", model="gemini-2.0-flash-001")
chat_model.invoke("こんにちは")

AIMessage(content='こんにちは！何かお手伝いできることはありますか？😊\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 1, 'candidates_token_count': 12, 'total_token_count': 13, 'prompt_tokens_details': [{'modality': 1, 'token_count': 1}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 12}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.06469776233037312, 'model_name': 'gemini-2.0-flash-001'}, id='run-039762e0-083f-41ef-9327-280f8a5e7536-0', usage_metadata={'input_tokens': 1, 'output_tokens': 12, 'total_tokens': 13})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(project="", model="gemini-2.0-flash-001") 
llm.invoke("こんにちは")

#Vertexai時の初期化パラメーターと異なり、GeminiAPI経由の場合はAPIキーがいりそう
#ただし、ここのOutputはChatModelクラスと違い、ただの文字列が返ってくる。

# 例: "こんにちは！何かお手伝いできることはありますか？😊\n"

# ChatPromptTemplateとPromptTemplateの違い

基本的な使い方

https://python.langchain.com/docs/concepts/prompt_templates/

In [64]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Any

## ChatPromptTemplate

from_messages

In [33]:
# systemプロンプトとuserプロンプトの設定
sys_prompt = "あなたは翻訳専門家です。{source_language} を {target_language} に翻訳する。"
user_prompt = """
[ariticle]
{article}
"""

# ChatPromptTemplate.from_messagesはBaseMessageを継承
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", sys_prompt),
    ("human", user_prompt),
  ]
)
prompt.format_messages(source_language="japnease", target_language="english", article="こんにちは")


[SystemMessage(content='あなたは翻訳専門家です。japnease を english に翻訳する。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n[ariticle]\nこんにちは\n', additional_kwargs={}, response_metadata={})]

### ChatPromptTemplateのインスタンス化

MessagesPlaceholderは複数チャット履歴またはHumanMessagesを複数与えときに役に立つ -> マルチモーダル時にtextと一緒に与える

In [78]:
# そのままインスタンス化する操作
# 複数チャット履歴またはHumanMessagesを複数与える -> マルチモーダル時にtextと一緒に与える

human_message_content = []
human_message_content.append(HumanMessage("おはよう"))
human_message_content.append(HumanMessage("写真です"))


prompt = ChatPromptTemplate(
  [
    SystemMessage(sys_prompt),
    MessagesPlaceholder("multimodalPlusUserInput")
  ],
  input_variables = ["source_language", "target_language"],
)

# TODO: source_languageとtarget_languageがうまくフォーマットされていない問題がある
# prompt.format_messages(source_language="japnease", target_language="english", multimodalPlusUserInput=human_message_content)
prompt.invoke({"source_language":"japanease", "target_language":"english", "multimodalPlusUserInput": human_message_content})

ChatPromptValue(messages=[SystemMessage(content='あなたは翻訳専門家です。{source_language} を {target_language} に翻訳する。', additional_kwargs={}, response_metadata={}), HumanMessage(content='おはよう', additional_kwargs={}, response_metadata={}), HumanMessage(content='写真です', additional_kwargs={}, response_metadata={})])

## PromptTemplate

In [16]:
prompt_template = PromptTemplate.from_template(
  "{name}に関する笑い話を教えて下さい。"
)

prompt_template.format(name="世界")

'世界に関する笑い話を教えて下さい。'

# Praser の使い方

In [17]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

In [80]:
class Joke(BaseModel):
  setup: str = Field(description="question")
  punchline: str = Field(description="answer")

joke_query = "ジョーク教えて下さい。"

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
  template = "答えてください。\n {format_instructions} \n {query} \n",
  # 毎回手動で与える必要のある変数
  input_variables=["query"],
  # templateに組み込まれている変数
  # get_format_instructionsはあくまでLangchain側で定義されているfew-short prompt
  partial_variables={"format_instructions":parser.get_format_instructions}
)

print(prompt.format(query=joke_query))

chain = prompt | chat_model | parser
chain.invoke({"query": joke_query})

答えてください。
 The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer", "type": "string"}}, "required": ["setup", "punchline"]}
``` 
 ジョーク教えて下さい。 



{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

# Stream

In [93]:
sys_prompt = ""
user_prompt = "ジョークを教えて下さい、200文字以上500文字以内。"

prompt = ChatPromptTemplate(
  [
  SystemMessage(sys_prompt),
  HumanMessage(user_prompt)
  ]
)

chain = prompt | chat_model

# 文字一つずつの出力方法
# TODO:もっといい方法ないかな?
for chunk in chain.stream({"input": ""}):
  for char in chunk.content:
    print(char, end="|", flush=True)

あ|る|と|こ|ろ|に|、|と|て|も|忘|れ|っ|ぽ|い|お|じ|い|さ|ん|が|い|ま|し|た|。|
|
|あ|る|日|、|お|じ|い|さ|ん|は|買|い|物|に|出|か|け|よ|う|と|し|ま|し|た|。|し|か|し|、|玄|関|で|立|ち|止|ま|り|、|何|を|買|う|か|全|く|思|い|出|せ|ま|せ|ん|。|
|
|困|っ|た|お|じ|い|さ|ん|は|、|奥|さ|ん|に|「|お|ば|あ|さ|ん|、|悪|い|ん|だ|け|ど|、|今|日|何|を|買|っ|て|く|る|ん|だ|っ|た|か|教|え|て|く|れ|な|い|か|？|」|と|尋|ね|ま|し|た|。|
|
|お|ば|あ|さ|ん|は|少|し|呆|れ|な|が|ら|も|「|あ|ら|、|ま|た|忘|れ|ち|ゃ|っ|た|の|？|い|い|わ|よ|。|卵|を|6|個|買|っ|て|き|て|ち|ょ|う|だ|い|。|も|し|パ|ン|屋|さ|ん|が|開|い|て|い|た|ら|、|パ|ン|も|6|個|買|っ|て|き|て|ね|」|と|言|い|ま|し|た|。|
|
|お|じ|い|さ|ん|は|何|度|も|「|卵|6|個|、|パ|ン|屋|さ|ん|が|あ|れ|ば|パ|ン|6|個|」|と|ブ|ツ|ブ|ツ|言|い|な|が|ら|家|を|出|ま|し|た|。|
|
|し|ば|ら|く|し|て|、|お|じ|い|さ|ん|は|買|い|物|か|ら|帰|っ|て|き|ま|し|た|。|し|か|し|、|手|に|持|っ|て|い|る|の|は|パ|ン|が|1|2|個|。|
|
|お|ば|あ|さ|ん|は|「|あ|ら|、|お|じ|い|さ|ん|。|卵|は|ど|こ|に|行|っ|た|の|？|そ|れ|に|、|ど|う|し|て|パ|ン|が|1|2|個|も|あ|る|の|？|」|と|尋|ね|ま|し|た|。|
|
|お|じ|い|さ|ん|は|ニ|ヤ|リ|と|笑|っ|て|「|パ|ン|屋|さ|ん|、|開|い|て|た|ん|だ|も|ん|！|」|
|

# Function Calling

# Cache

In [96]:
chat_model.invoke("こんにちは").usage_metadata

{'input_tokens': 1, 'output_tokens': 12, 'total_tokens': 13}

# Langsmith

openaiのモデルしか対応してない?

In [100]:
import os

os.environ["LANGSMITH_API_KEY"] =""
os.environ["LANGSMITH_TRACING"] ="true"
os.environ["LANGSMITH_ENDPOINT"] =""
os.environ["LANGSMITH_PROJECT"] =""

chat_model.invoke("こんにちは")

AIMessage(content='こんにちは！何かお手伝いできることはありますか？\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 1, 'candidates_token_count': 11, 'total_token_count': 12, 'prompt_tokens_details': [{'modality': 1, 'token_count': 1}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.12914298881183972, 'model_name': 'gemini-2.0-flash-001'}, id='run-24618940-ab52-45e4-9665-3bf1464eca52-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12})